# Выбор региона для бурения нефтескважины

# 1. Загрузка и подготовка данных

In [52]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from numpy.random import RandomState
state = np.random.RandomState(12345)

data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_0.head(5))
print()
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_1.head(5))
print()
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_2.head(5))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [53]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Для анализа нам даны 3 датасета, содержащие по 100 тыс. записей о месторождениях нефти в регионе. Данные полные, не содержат пропусков. Каждый датасет имеет столбец id, содержащий информацию вещественного типа об идентификаторе месторождения, для построения модели эта информация не потребуется, поэтому во всех трех датасетах удалим этот столбец.

In [54]:
del data_0['id']
del data_1['id']
del data_2['id']

# 2. Обучение и проверка модели

##### Разобьём данные на обучающую и валидационную выборки

Разделим исходные данные на на выборки в следующем процентном соотношении:
* data_train - обучающая выборка (75%)
* data_valid - валидационная аыборка (25%)

In [55]:
data_0_train, data_0_valid = train_test_split(data_0, test_size=0.25, random_state=12345)
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)

Сохраним признаки в отдельных переменных по каждому датасету: features - все признаки, кроме целевого признака (объём запасов в месторождении (тыс. баррелей)), target — целевой признак, столбец product.

In [56]:
features_train_0 = data_0_train.drop(columns=['product'], axis=1)
target_train_0 = data_0_train['product']
features_train_1 = data_1_train.drop(columns=['product'], axis=1)
target_train_1 = data_1_train['product']
features_train_2 = data_2_train.drop(columns=['product'], axis=1)
target_train_2 = data_2_train['product']

features_valid_0 = data_0_valid.drop(columns=['product'], axis=1)
target_valid_0 = data_0_valid['product']
features_valid_1 = data_1_valid.drop(columns=['product'], axis=1)
target_valid_1 = data_1_valid['product']
features_valid_2 = data_2_valid.drop(columns=['product'], axis=1)
target_valid_2 = data_2_valid['product']

##### Обучим модель и сделаем предсказания на валидационной выборке

In [57]:
def model_lr(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)
    print('Средний запас сырья:', prediction_valid.mean())
    print('RMSE:', mean_squared_error(target_valid, prediction_valid)**0.5)
    print('R2:', r2_score(target_valid, prediction_valid))
    print('MAE:', mean_absolute_error(target_valid, prediction_valid))
    return prediction_valid, target_valid

In [58]:
print('Region 0')
prediction_valid_0, target_valid_0 = model_lr(features_train_0, features_valid_0, target_train_0, target_valid_0)

Region 0
Средний запас сырья: 92.59256778438038
RMSE: 37.5794217150813
R2: 0.27994321524487786
MAE: 30.919600777151313


In [59]:
print('Region 1')
prediction_valid_1, target_valid_1 = model_lr(features_train_1, features_valid_1, target_train_1, target_valid_1)

Region 1
Средний запас сырья: 68.728546895446
RMSE: 0.893099286775616
R2: 0.9996233978805127
MAE: 0.718766244212475


In [60]:
print('Region 2')
prediction_valid_2, target_valid_2 = model_lr(features_train_2, features_valid_2, target_train_2, target_valid_2)

Region 2
Средний запас сырья: 94.96504596800489
RMSE: 40.02970873393434
R2: 0.20524758386040443
MAE: 32.792652105481814


Анализируя средний запас на месторождения по регионам можно сказать, что наибольшее значение у Region_2 равное примерно 95 тыс. баррелей на месторождение. Если обращаться к другим метрикам, то смотря на среднеквадратическое отклонение (RMSE) можно отметить, что больший разброс по значению от среднего наблюдается у Region_2 и Region_0 - примено 38-39 тыс. баррелей. Что касаемо Region_1, то там данный показатель на уровне меньше 1 тысячи баррелей, значит, запасы нефти на месторождениях в нем примерно на одном уровне. 

Рассматривая коэффициент детерминации (R2), который показывает качество предсказаний модели, лучший показатель у Region_1, близкий к единице, это означает, что такая модель предсказывает все ответы идеально. Остальные регионы с метрикой на уровне 0,2-0,27, что говорит о том, что модель предсказывает правильные ответы чуть более лучше, чем средняя.

# 3. Подготовка к расчёту прибыли

##### Посчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки

3.1. Сохраните в коде все ключевые значения для расчётов.

3.2. Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.

3.3. Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [61]:
BUDGET = 10000000000
BUDGET_FOR_ONE_WELL = 50000000
ONE_BARREL_VALUE = 4500

q_well = BUDGET / BUDGET_FOR_ONE_WELL
q_barrel = (BUDGET_FOR_ONE_WELL / ONE_BARREL_VALUE) / 1000

print('Количество скважин в рамках бюджета:', q_well)
print('Минимальное количество баррелей для нулевой рентабельности:', q_barrel)

Количество скважин в рамках бюджета: 200.0
Минимальное количество баррелей для нулевой рентабельности: 11.11111111111111


Имеющихся денежных ресурсов компании хватит на бурение 200 скважин. Если оценивать сколько необходимо добыть баррелей нефти для того, чтобы операционная прибыль по скважине была равной нулю, то это значение равно примерно 11,1 тыс. баррелей при цене 4500 рублей за баррель. Нам видится, что данное значение достижимо в среднем для всех регионов (среднее запасов в разы больше 11 тыс. баррелей)

##### Создадим функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели

In [62]:
def revenue(target_subsample, probs_subsample, count):
    prediction = probs_subsample.sort_values(ascending = False).head(count)
    selected = target_subsample[prediction.index] * 1000 * ONE_BARREL_VALUE
    return selected.sum() - (BUDGET_FOR_ONE_WELL * count)

# 4. Расчёт прибыли и рисков 

##### Применим технику Bootstrap, чтобы найти распределение прибыли

In [63]:
def bootstrap (target_valid, prediction_valid):
    
    values = []

    for i in range(1000):
        target_valid = pd.Series(target_valid)
        prediction_valid = pd.Series(prediction_valid)
        target_subsample = target_valid.sample(n=500, replace=True, random_state=state)
        prediction_subsample = prediction_valid[target_subsample.index]
        values.append(revenue(target_subsample, prediction_subsample, 200))

    values = pd.Series(values)    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    revenue_mean = values.mean()
    values = values.sort_values(ascending = False).head(200)

    print('Средняя выручка:', revenue_mean)
    print('2.5%-квантиль:', lower)
    print('97.5%-квантиль:', upper)
    return  lower, upper, revenue_mean, values

##### Найдем среднюю прибыль, 95%-й доверительный интервал для каждого региона, а также оценим риски убытков

In [64]:
lower_0, upper_0, revenue_mean_0, values_0 = bootstrap(target_valid_0, prediction_valid_0)
print ("Риск убыточности:", stats.percentileofscore(values_0, 0))

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Средняя выручка: 74428833535.76396
2.5%-квантиль: 67929222865.43791
97.5%-квантиль: 80494545871.98251
Риск убыточности: 0.0


In [65]:
lower_1, upper_1, revenue_mean_1, values_1 = bootstrap(target_valid_1, prediction_valid_1)
print ("Риск убыточности:", stats.percentileofscore(values_1, 0))

Средняя выручка: 53109372601.8785
2.5%-квантиль: 47099633430.93045
97.5%-квантиль: 59340296245.5019
Риск убыточности: 0.0


In [66]:
lower_2, upper_2, revenue_mean_2, values_2 = bootstrap(target_valid_2, prediction_valid_2)
print ("Риск убыточности:", stats.percentileofscore(values_2, 0))

Средняя выручка: 76860166312.80507
2.5%-квантиль: 70637189055.62447
97.5%-квантиль: 83166998307.12842
Риск убыточности: 0.0


##### Вывод

Проанализировав запасы нефти на месторождениях в трех регионах можно отметить следующее. При оценке средних объемов добычи и среднеквадратического отклонения решили, что наиболее лучшие показатели у Region_0 и Region_2. Средние запасы более 90 тыс баррелей на одно месторождение и примерно одинаковое среднеквадратическое отклонение (большое, но примерно одинаковое). Для того, чтобы понять какую модель необходимо рекомендовать провели бутстреп-анализ. Средняя прибыль выше в третьем анализируемом регионе Region_2 с доверительным интревалом от 77,1 до 83,4 млрд рублей с одного региона по 200 самым лучшим месторождениям. Рассичтанный риск безубыточности нулевой у всех трех регионов, поэтому выбор останавливаем на Region_2.